In [2]:
# INSECURE: example produced by an AI (do NOT use in production)
users = {
    "alice": "alicepwd123",     # plaintext password stored in code
    "bob": "bobpassword"
}

def login(username, password):
    # Hardcoded credential check
    if username in users and users[username] == password:
        return True
    return False

# Bad: prints and returns sensitive info
print(login("alice", "alicepwd123"))

True


In [3]:
# secure_login.py
import os
import bcrypt
import time
from getpass import getpass

# Example storage: in real systems use a database (and never commit secrets).
# For demo: load user -> hashed_password mapping from a file or secure store.
# Here we load from an environment variable to avoid hardcoding:
# Example env var: export USER_ALICE_HASH=b'$2b$12$...'
def load_user_hash(username: str) -> bytes | None:
    # Expect env var like USER_<USERNAME>_HASH
    key = f"USER_{username.upper()}_HASH"
    val = os.getenv(key)
    return val.encode() if val else None

def hash_password(plain_password: str) -> bytes:
    # bcrypt default salt + cost is safe for many use-cases
    return bcrypt.hashpw(plain_password.encode('utf-8'), bcrypt.gensalt(rounds=12))

def verify_password(stored_hash: bytes, password_attempt: str) -> bool:
    # bcrypt.checkpw is safe and uses constant-time comparison internally
    try:
        return bcrypt.checkpw(password_attempt.encode('utf-8'), stored_hash)
    except ValueError:
        # malformed hash
        return False

# Simple in-process rate limiter (placeholder). Use redis or other store in production.
_failed_attempts = {}
LOCKOUT_THRESHOLD = 5
LOCKOUT_WINDOW = 300  # seconds

def is_locked(username: str) -> bool:
    rec = _failed_attempts.get(username)
    if not rec:
        return False
    attempts, first_ts = rec
    if attempts >= LOCKOUT_THRESHOLD and (time.time() - first_ts) < LOCKOUT_WINDOW:
        return True
    if (time.time() - first_ts) >= LOCKOUT_WINDOW:
        # reset window
        _failed_attempts.pop(username, None)
        return False
    return False

def register_failed_attempt(username: str):
    rec = _failed_attempts.get(username)
    if not rec:
        _failed_attempts[username] = [1, time.time()]
    else:
        rec[0] += 1

def login_flow():
    username = input("Username: ").strip()
    if is_locked(username):
        print("Account temporarily locked due to repeated failures. Try later.")
        return False

    # secure prompt (no echo)
    password = getpass("Password: ")

    stored_hash = load_user_hash(username)
    if stored_hash is None:
        # Generic message to avoid user enumeration
        print("Invalid credentials")
        register_failed_attempt(username)
        return False

    if verify_password(stored_hash, password):
        print("Login successful")
        # reset failed attempts on success
        _failed_attempts.pop(username, None)
        return True
    else:
        print("Invalid credentials")
        register_failed_attempt(username)
        return False

if __name__ == "__main__":
    print("Run this demo after placing hashed passwords in environment variables.")
    print("Do not store real production secrets in env vars for long-term; use a secrets manager.")
    login_flow()

Run this demo after placing hashed passwords in environment variables.
Do not store real production secrets in env vars for long-term; use a secrets manager.
Invalid credentials
